In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 10 06:03:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture

!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate
import os
import re
import wandb
import torch
import numpy as np
import json
from evaluate import load
from datasets import load_dataset, Audio
from transformers import EarlyStoppingCallback
from huggingface_hub import HfFolder

In [ ]:
from huggingface_hub import HfFolder
token = os.getenv("HF_TOKEN")  # Use env variable instead of hardcoding Hugging Face tokenSXwdwQiQBwpzvZdjPObRIxFxMSQWspDVFJ"
HfFolder.save_token(token)

In [ ]:
from datasets import load_dataset, DatasetDict

test5hr = load_dataset("KYAGABA/oromo_cleaned_dataset")['test']
print(test5hr)

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'text', 'speaker', 'split'],
    num_rows: 1103
})


In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['text'] = [text.lower() for text in batch['text']]
    batch['text'] = [unicodedata.normalize('NFKC', text) for text in batch['text']]
    batch['text'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['text']]
    batch['text'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['text']]
    batch['text'] = [re.sub(r'_', ' ', text) for text in batch['text']]
    batch['text'] = [' '.join(text.split()) for text in batch['text']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

In [ ]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = [
    "asr-africa/whisper-small-Sagalee-orm-85hrs-1"
]

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    return wer, cer

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)
    results = []
    all_predictions = []
    all_references = []

    for audio, text in tqdm(zip(test5hr["audio"], test5hr["text"]),
                                                         total=test5hr.num_rows, desc="Running inference"):
        prediction = pipe(audio)['text']
        wer, cer = compute_individual_metrics(prediction, text)

        results.append({
            "Prediction": prediction,
            "Reference": text,
            "WER": wer,
            "CER": cer,
            # "Source": source
        })
        all_predictions.append(prediction)
        all_references.append(text)

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    # average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    # print("\nAverage WER and CER by Source:")
    # print(average_metrics_by_source)
    # Save detailed results to CSV
    output_filename = f'texts_{model_name.split("/")[-1]}.csv'
    # output_filename = f"{model_name}_detailed_results.csv"
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")

# Loop through each model and evaluate
for model in models:
    run_inference_for_model(model)

Processing model: asr-africa/whisper-small-Sagalee-orm-85hrs-1


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Running inference:   0%|          | 0/1103 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Running inference: 100%|██████████| 1103/1103 [06:00<00:00,  3.06it/s]

Calculated overall WER: 0.5082533265959238
Calculated overall CER: 0.43171551156613924
Results saved to texts_whisper-small-Sagalee-orm-85hrs-1.csv


In [ ]:
from transformers import AutoProcessor, pipeline

model_name = "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1"

# Load the processor/tokenizer explicitly
processor = AutoProcessor.from_pretrained(model_name)

# Pass the processor to the pipeline
pipe = pipeline("automatic-speech-recognition", model=model_name, tokenizer=processor.tokenizer, device=device)

In [ ]:
from transformers import AutoTokenizer

model_name = "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1"

# Try loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.vocab_size)  # Check if the vocabulary is properly loaded

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

269


In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm

pipe = pipeline("automatic-speech-recognition",model= "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1", device =device)

In [ ]:
def transcribe(audio):
    text = pipe(audio)["text"]
    return text

In [ ]:
predictions = [prediction['text'] for prediction in tqdm(pipe(KeyDataset(test5hr, "audio"), batch_size=32),\
                                                              desc= "Running inference", total= test5hr.num_rows)]

In [ ]:
test5hr = test5hr.add_column("predictions", predictions)

In [ ]:
print("Test WER: {:.4f}".format(wer_metric.compute(predictions=test5hr["predictions"], references=test5hr["text"])))
print("Test CER: {:.4f}".format(cer_metric.compute(predictions=test5hr["predictions"], references=test5hr["text"])))

In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = [
    "asr-africa/facebook-mms-1b-all-common_voice_fleurs-amh-200hrs-v1"
]

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    return wer, cer

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device, return_timestamps='word')
    results = []
    all_predictions = []
    all_references = []

    for audio, text in tqdm(zip(test5hr["audio"], test5hr["text"]),
                                                         total=test5hr.num_rows, desc="Running inference"):
        prediction = pipe(audio)['text']
        wer, cer = compute_individual_metrics(prediction, text)

        results.append({
            "Prediction": prediction,
            "Reference": text,
            "WER": wer,
            "CER": cer,
            # "Source": source
        })
        all_predictions.append(prediction)
        all_references.append(text)

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    # average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    # print("\nAverage WER and CER by Source:")
    # print(average_metrics_by_source)
    # Save detailed results to CSV
    output_filename = f'texts_{model_name.split("/")[-1]}.csv'
    # output_filename = f"{model_name}_detailed_results.csv"
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")

# Loop through each model and evaluate
for model in models:
    run_inference_for_model(model)


In [ ]:
import os
import json
import re
import torch
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio, concatenate_datasets
from transformers import Wav2Vec2BertProcessor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
import evaluate
from huggingface_hub import HfApi

# Initialize WER and CER metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# Model checkpoint and repository information
model_checkpoint =  "asr-africa/whisper-small-sagale_dataset-ormo-5hrs-v2"
repo_id = model_checkpoint

# Define the alphabet for Amharic and create vocab.json
alphabet = 'ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቋበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኋነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኢኣኤእኦከኩኪካኬክኮኳኸኹኺኻኼኽኾዃወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮደዱዲዳዴድዶዷጀጁጂጃጄጅጆጇገጉጊጋጌግጎጓጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗ '
alphabet = sorted(list(set(alphabet)))
vocab_dict = {v: k for k, v in enumerate(alphabet)}
vocab_dict["|"] = len(vocab_dict)  # Word delimiter token
vocab_dict["[UNK]"] = len(vocab_dict)  # Unknown token
vocab_dict["[PAD"] = len(vocab_dict)  # Padding token
vocab_file_path = "vocab.json"
with open(vocab_file_path, "w") as vocab_file:
    json.dump(vocab_dict, vocab_file)
from transformers import WhisperTokenizer
# Create and/or load the tokenizer
tokenizer = WhisperTokenizer.from_pretrained(model_checkpoint)
print("Loaded tokenizer from model checkpoint.")
# except:
#     tokenizer = Wav2Vec2CTCTokenizer(vocab_file=vocab_file_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
#     tokenizer.push_to_hub(repo_id, commit_message="Adding custom tokenizer and vocabulary")
#     print("Created and pushed custom tokenizer.")

# Push vocab.json to the Hub explicitly
api = HfApi()
api.upload_file(path_or_fileobj=vocab_file_path, path_in_repo="vocab.json", repo_id=repo_id, commit_message="Adding vocabulary file")
from transformers import WhisperProcessor
from transformers import WhisperForConditionalGeneration
# Load the processor and model
processor = WhisperProcessor.from_pretrained(model_checkpoint, tokenizer=tokenizer)
model =WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define Audio feature with consistent sampling rate
audio_feature = Audio(sampling_rate=16000, mono=True)

# Define data preprocessing and helper functions
def unify_and_add_source(batch, dataset_name):
    batch['sentence'] = batch.get('text', batch.get('sentence', ""))
    batch['dataset_source'] = dataset_name
    return batch

def preprocess_text(batch):
    batch['sentence'] = re.sub(r'[^\w\s\'፡።፣፤፥፦፨]', '', batch['sentence'].lower()).strip()
    return batch

def get_lens(batch):
    try:
        audio_len = librosa.get_duration(y=batch['audio']['array'], sr=batch['audio']['sampling_rate'])
    except:
        audio_len = 0.0
    transcript_len = len(batch['sentence'])
    batch.update({
        'audio_len': audio_len,
        'transcript_len': transcript_len,
        'len_ratio': audio_len / transcript_len if transcript_len > 0 else 0,
        'num_feature_vecs': int(np.round(audio_len * 1000 / 20))
    })
    return batch

def data_checks(batch):
    audio_check = 1.0 < batch['audio_len'] < 30.0
    transcript_check = batch['transcript_len'] > 10
    input_output_ratio_check = (batch['num_feature_vecs'] / batch['transcript_len']) > 1.0 if batch['transcript_len'] > 0 else False
    return audio_check and transcript_check and input_output_ratio_check

def select_audio_by_hours(dataset, target_duration=4500):
    cumulative_duration, selected_indices = 0.0, []
    for idx, audio_data in enumerate(dataset["audio"]):
        duration = librosa.get_duration(y=audio_data["array"], sr=audio_data["sampling_rate"])
        cumulative_duration += duration
        if cumulative_duration > target_duration:
            break
        selected_indices.append(idx)
    return dataset.select(selected_indices)

# Dataset details with split configuration
datasets_info = {

    "AMHARIC_DATASET": {"path": "KYAGABA/amharic_cleaned_testset_verified", "split": "val"}
}

# Inference function
def run_inference_on_dataset(dataset, dataset_name):
    predictions, references = [], []
    for batch in dataset:
        inputs = torch.tensor([batch["input_values"]]).to(device)
        with torch.no_grad():
            logits = model(inputs).logits
        pred_ids = torch.argmax(logits, dim=-1)
        predictions.append(processor.batch_decode(pred_ids)[0])
        references.append(batch["sentence"])
    wer = wer_metric.compute(predictions=predictions, references=references)
    cer = cer_metric.compute(predictions=predictions, references=references)
    print(f"Results for {dataset_name} - WER: {wer}, CER: {cer}")
    return {"Dataset": dataset_name, "WER": wer, "CER": cer, "Predictions": predictions, "References": references}

# Process each dataset individually
processed_datasets, all_results = [], []
for dataset_name, info in datasets_info.items():
    dataset_path, split = info["path"], info["split"]
    dataset = load_dataset(dataset_path, split=split).cast_column("audio", audio_feature)
    dataset = dataset.map(lambda batch: unify_and_add_source(batch, dataset_name)).map(preprocess_text)
    dataset = dataset.map(get_lens).filter(data_checks)
    dataset = select_audio_by_hours(dataset, target_duration=4500)
    dataset = dataset.map(lambda batch: {"input_features": processor(audio=batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_values[0]})
    result = run_inference_on_dataset(dataset, dataset_name)
    all_results.append(result)
    processed_datasets.append(dataset)

# Combined dataset inference
combined_test_dataset = concatenate_datasets(processed_datasets)
combined_result = run_inference_on_dataset(combined_test_dataset, "Combined Test Set")
all_results.append(combined_result)
# Log results to CSV
def log_results_to_csv(results, output_path="inference_results.csv"):
    rows = [{"Dataset": res["Dataset"], "Original Transcript": ref, "Predicted Transcript": pred, "WER": res["WER"], "CER": res["CER"]}
            for res in results for pred, ref in zip(res["Predictions"], res["References"])]
    pd.DataFrame(rows).to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

log_results_to_csv(all_results)

Loaded tokenizer from model checkpoint.


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Map:   0%|          | 0/486 [00:00<?, ? examples/s]

AttributeError: 